In [1]:
# coding=utf-8
from __future__ import print_function, absolute_import 
from gm.api import *

In [2]:
import numpy as np
import pandas as pd

In [3]:
set_token('9c0950e38c59552734328ad13ad93b6cc44ee271')
# 基础交易数据

In [4]:
# context.symbol = 'SHSE.510300'
start_date = '2019-02-28'
end_date =  '2020-02-28'

In [6]:
trade_data = history('SHSE.510300', frequency='1d', start_time=start_date, end_time=end_date, fill_missing='last',
                         df=True).set_index('eob')

In [7]:
trade_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 244 entries, 2019-02-28 00:00:00+08:00 to 2020-02-28 00:00:00+08:00
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype                        
---  ------     --------------  -----                        
 0   symbol     244 non-null    object                       
 1   frequency  244 non-null    object                       
 2   open       244 non-null    float64                      
 3   high       244 non-null    float64                      
 4   low        244 non-null    float64                      
 5   close      244 non-null    float64                      
 6   volume     244 non-null    int64                        
 7   amount     244 non-null    float64                      
 8   pre_close  244 non-null    float64                      
 9   position   244 non-null    int64                        
 10  bob        244 non-null    datetime64[ns, Asia/Shanghai]
dtypes: datetime64[ns, Asia/Shanghai](1)

# 添加输出/监督变量：未来T日的收益率

In [8]:
def add_return_column(trade_data, period=3):
    """
    向DataFrame添加未来T日的收益率。

    参数:
    - trade_data: 包含交易数据的DataFrame。
    - period: 未来的天数，默认为3。

    返回值:
    - 该函数没有返回值，但会修改传入的DataFrame，为每个交易日添加一个新列，表示未来T日的收益率。
    """
    
    #对数化收益率 https://uestc.feishu.cn/wiki/DrMYw98b2iVGRbkBfH3cGzeTnoe#MtK2dQhZNoTfuqxKECbcMElVnKd
    trade_data['log_return'] = np.log(trade_data['close'] / trade_data['close'].shift(period))
    #清除shift后的有空白数据的行
    trade_data.dropna(inplace=True)


In [9]:
add_return_column(trade_data)
trade_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 241 entries, 2019-03-05 00:00:00+08:00 to 2020-02-28 00:00:00+08:00
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype                        
---  ------      --------------  -----                        
 0   symbol      241 non-null    object                       
 1   frequency   241 non-null    object                       
 2   open        241 non-null    float64                      
 3   high        241 non-null    float64                      
 4   low         241 non-null    float64                      
 5   close       241 non-null    float64                      
 6   volume      241 non-null    int64                        
 7   amount      241 non-null    float64                      
 8   pre_close   241 non-null    float64                      
 9   position    241 non-null    int64                        
 10  bob         241 non-null    datetime64[ns, Asia/Shanghai]
 11  log_return  241 non-nu

# 添加技术指标

In [10]:
def add_ma_columns(trade_data, ma_periods):
    """
    向DataFrame添加移动平均列。

    参数:
    - trade_data: 包含交易数据的DataFrame。
    - ma_periods: 一个包含要计算的移动平均周期的列表，每个元素是一个整数，表示天数。

    返回值:
    - 该函数没有返回值，但会修改传入的DataFrame，为每个指定的周期添加一个新列。
    """
    for period in ma_periods:
        ma_column_name = f'MA_{period}'  # 根据周期生成列名称，如'MA_5'表示5天移动平均
        trade_data[ma_column_name] = trade_data['close'].rolling(window=period).mean()

def add_ema_columns(trade_data, ema_periods):
    """
    向DataFrame添加指数移动平均列。

    参数:
    - trade_data: 包含交易数据的DataFrame。
    - ema_periods: 一个包含要计算的指数移动平均周期的列表，每个元素是一个整数，表示天数。

    返回值:
    - 该函数没有返回值，但会修改传入的DataFrame，为每个指定的周期添加一个新列。
    """
    for period in ema_periods:
        ema_column_name = f'EMA_{period}'  # 根据周期生成列名称，如'EMA_5'表示5天指数移动平均
        trade_data[ema_column_name] = trade_data['close'].ewm(span=period).mean()

def add_rsi_factor(trade_data, period=14):
    """
    向DataFrame添加相对强弱指数（RSI）因子。

    参数:
    - trade_data: 包含交易数据的DataFrame。
    - period: 计算RSI的周期，默认为14天。
    """
    delta = trade_data['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    trade_data['RSI'] = 100 - (100 / (1 + rs))

def add_atr_factor(trade_data, period=14):
    """
    向DataFrame添加平均真实范围（ATR）因子。

    参数:
    - trade_data: 包含交易数据的DataFrame。
    - period: 计算ATR的周期，默认为14天。
    """
    high_low = trade_data['high'] - trade_data['low']
    high_close = (trade_data['high'] - trade_data['close'].shift()).abs()
    low_close = (trade_data['low'] - trade_data['close'].shift()).abs()
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = ranges.max(axis=1)
    trade_data['ATR'] = true_range.rolling(window=period).mean()

def add_bollinger_band_width_factor(trade_data, period=20, num_std=2):
    """
    向DataFrame添加布林带宽度因子。

    参数:
    - trade_data: 包含交易数据的DataFrame。
    - period: 计算布林带的周期，默认为20天。
    - num_std: 布林带的标准差倍数，默认为2。
    """
    ma = trade_data['close'].rolling(window=period).mean()
    std = trade_data['close'].rolling(window=period).std()
    upper_band = ma + (std * num_std)
    lower_band = ma - (std * num_std)
    trade_data['Bollinger_Width'] = upper_band - lower_band

def add_vwap_factor(trade_data):
    """
    向DataFrame添加成交量加权平均价格（VWAP）因子。

    参数:
    - trade_data: 包含交易数据的DataFrame。
    """
    trade_data['VWAP'] = (trade_data['volume'] * trade_data['close']).cumsum() / trade_data['volume'].cumsum()


In [11]:
add_ma_columns(trade_data, [5, 10, 20, 60, 120])
add_ema_columns(trade_data, [5, 10, 20, 60, 120])
add_rsi_factor(trade_data)
add_atr_factor(trade_data)
add_bollinger_band_width_factor(trade_data)
add_vwap_factor(trade_data)
trade_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 241 entries, 2019-03-05 00:00:00+08:00 to 2020-02-28 00:00:00+08:00
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype                        
---  ------           --------------  -----                        
 0   symbol           241 non-null    object                       
 1   frequency        241 non-null    object                       
 2   open             241 non-null    float64                      
 3   high             241 non-null    float64                      
 4   low              241 non-null    float64                      
 5   close            241 non-null    float64                      
 6   volume           241 non-null    int64                        
 7   amount           241 non-null    float64                      
 8   pre_close        241 non-null    float64                      
 9   position         241 non-null    int64                        
 10  bob              241 non-

# 宏观数据指标

# 情绪数据

In [18]:
def fetch_fear_greed_index():
    """
    获取恐慌与贪婪指数。

    返回:
    - DataFrame: 包含恐慌与贪婪指数的DataFrame。
    """
    index_df = ak.index_crypto_feargreed()
    return index_df

In [ ]:
def calculate_ar(data, period=26):
    """
    计算AR（人气指标）。

    参数:
    - data: 包含'high', 'low', 'open'列的DataFrame。
    - period: 计算指标的周期，默认为26天。

    返回:
    - ar: DataFrame，包含计算周期内的AR值。
    """
    ar_numerator = (data['high'] - data['open']).rolling(window=period).sum()
    ar_denominator = (data['open'] - data['low']).rolling(window=period).sum()
    ar = (ar_numerator / ar_denominator) * 100
    return ar


In [ ]:
def calculate_br(data, period=26):
    """
    计算BR（意愿指标）。

    参数:
    - data: 包含'high', 'low', 'close'列的DataFrame。
    - period: 计算指标的周期，默认为26天。

    返回:
    - br: DataFrame，包含计算周期内的BR值。
    """
    br_numerator = (data['high'] - data['close'].shift()).rolling(window=period).sum()
    br_denominator = (data['close'].shift() - data['low']).rolling(window=period).sum()
    br = (br_numerator / br_denominator) * 100
    return br


# 特征工程

## 数据整合